In [1]:
from confluent_kafka import Consumer, KafkaError
from fastavro import reader
import io
import json
import random
import settings

In [2]:
def make_stamp_annotation(record):
    r = {}
    classdict = {}
    maxprob = 0
    for k,v in record['probabilities'].items():
        classdict[k] = float('%.3f'%v)
        if v > maxprob:
            r['classification'] = k
            maxprob = v
    r['objectId']       = record['objectId']
    r['classdict']      = classdict
    if r['classification'] in ['VS', 'AGN', 'asteroid', 'bogus']:
        return None
    else:
        return r

In [3]:
def make_lc_annotation(record):
    r = {}
    r['objectId'] = record['oid']
    lcc = record['lc_classification']
    r['classification'] = lcc['class']
    classdict = {}
    for k,v in lcc['probabilities'].items():
        if v > 0.02:
            classdict[k] = float('%.3f'%v)
    r['classdict'] = classdict
    if r['classification'] in ['E', 'Periodic-Other']:
        return None
    else:
        return r

In [4]:
#print(settings.ALERCE_KAFKA)
#print(settings.ALERCE_GROUP_ID)
#print(settings.ALERCE_PASSWORD)

In [5]:
conf = {
    'bootstrap.servers': settings.ALERCE_KAFKA,
    'group.id'         : settings.ALERCE_GROUP_ID,
    'security.protocol': 'SASL_SSL',
    'sasl.mechanism'   : 'SCRAM-SHA-512',
    'sasl.username'    : 'lasair',
    'sasl.password'    : settings.ALERCE_PASSWORD,
    'auto.offset.reset': 'earliest',
}
streamReader = Consumer(conf)
t = list(streamReader.list_topics().topics.keys())
print('Topics are ', t)

Topics are  ['lc_classifier_ztf_20240602', 'lc_classifier_ztf__ATAT_forced_phot_20240826', 'lc_classifier_ztf_20240613', 'stamp_classifier_20240522', 'lc_classifier_ztf__BHRF_forced_phot_20240826', 'lc_classifier_ztf_20240429', 'lc_classifier_ztf_20240903', 'lc_classifier_ztf_20240526', 'lc_classifier_ztf_20240807', 'lc_classifier_ztf_20240706', 'lc_classifier_ztf_20240530', 'lc_classifier_ztf_20240630', 'stamp_classifier_20240811', 'stamp_classifier_20240817', 'stamp_classifier_20240822', 'lc_classifier_ztf_20240418', 'lc_classifier_ztf_20240717', 'stamp_classifier_20240719', 'lc_classifier_ztf_20240519', 'stamp_classifier_20240907', 'lc_classifier_ztf__BHRF_forced_phot_20240908', 'stamp_classifier_20240806', 'lc_classifier_ztf__BHRF_forced_phot_20240901', 'lc_classifier_ztf_20240906', 'stamp_classifier_20240904', 'lc_classifier_ztf_20240822', 'stamp_classifier_20240508', 'stamp_classifier_20240830', 'lc_classifier_ztf_20240714', 'lc_classifier_ztf__BHRF_forced_phot_20240822', 'lc_cla